# <span class="anchor" id="top">Initial Grid Setup</span>

#### Using [modflow-setup](https://github.com/aleaf/modflow-setup/tree/develop/mfsetup) it is possible to build only the discretization package to evaluate grid options. In the following script

1. [Initial imports](#intro)<br>
2. [Bounding Box and Grid Definition](#bound)<br>
3. [Modify the .yml File](#yml)<br>
4. [Create the DIS File](#dis)<br>
5. [Create a shapefile of the Mahomet Boundary](#shapefile)<br>

## <span class="anchor" id="intro">1. Initial imports</span>
[Return to Top](#top)<br>

In [1]:
import sys
sys.path.append('../python_packages_static')
import geopandas as gp
import flopy as fp
import mfsetup
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml
import pyproj



#### We need to obtain the base directory. This is essential because MODFLOW setup leaves us in the working model directory. Note that if the MODFLOW setup code fails, the current working directory might be in the temp folder, and you will need to restart the kernel to reset the current working directory. 

In [2]:
basedir = os.getcwd()
basedir

'C:\\Users\\dbabrams\\Documents\\ISWS\\GitHub\\neversink_workflow_ISWSfork\\notebooks_preprocessing_complete - Mahomet'

## <span class="anchor" id="bound">2. Bounding Box and Grid Definition</span>
[Return to Top](#top)<br>

#### Create a bounding box using a shapefile that will define the model extent. Here, we are just recreating the Mahomet Aquifer model, so we simply use a dissolve of the model grid shapefile as our input. 

In [3]:
domain = gp.read_file('../source_data/shapefiles/Extents/mahomet_dissolve.shp')

#### Now we obtain the CRS, which we need for MODFLOW-SETUP. Please note that in the Neversink version of this code, they read in an EPSG, but our coordinate system does not have an analogous EPSG. At least at this point in our analyses, it only appears to be the CRS that matters. The example.yml has been updated to read in a CRS instead of an EPSG.

In [4]:
#unlike neversink, we extract the crs directly and read that in rather than reading in the epsg in MODFLOW setup
crs = pyproj.CRS(domain.crs)
print(crs)

PROJCRS["Clarke_1866_Lambert_Conformal_Conic",BASEGEOGCRS["NAD27",DATUM["North American Datum 1927",ELLIPSOID["Clarke 1866",6378206.4,294.9786982,LENGTHUNIT["metre",1]],ID["EPSG",6267]],PRIMEM["Greenwich",0,ANGLEUNIT["Degree",0.0174532925199433]]],CONVERSION["unnamed",METHOD["Lambert Conic Conformal (2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false origin",33,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude of false origin",-89.5,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude of 1st standard parallel",33,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude of 2nd standard parallel",45,ANGLEUNIT["Degree",0.0174532925199433],ID["EPSG",8824]],PARAMETER["Easting at false origin",2999994,LENGTHUNIT["US survey foot",0.304800609601219],ID["EPSG",8826]],PARAMETER["Northing at false origin",0,LENGTHUNIT["US survey foot",0.304800609601219],ID["EPSG",8827]]],CS[Cartesian,2],AXIS["easting",east,ORDER[1],LENGTHUNIT["

#### With the crs, we can explore and extract the geometry of the model. Specifically, we need to extract the total distance in the x and y directions to be able to calculate the number of cells and columns later on. 

In [5]:
# plot the model domain. Since this is a shapefile with a single polygon, the entire domain geometry is plotted below
domain.geometry[0]

# extract the model bounds
bounds = domain.geometry[0].bounds
print(bounds)

# define the model domain length and height
tot_xdist, tot_ydist = bounds[2]-bounds[0], bounds[3]-bounds[1]
print (tot_xdist,tot_ydist)

(2751120.000000003, 2494080.000000006, 3547079.9999999944, 2857080.000000006)
795959.9999999916 363000.0


#### Define our grid spacing and the number of cells in the x and y direction.

### Important: we are not snapping to the national grid at this point in time. As currently written, we would need to set dx = 1000 ft or a value that is divisible into 1000 (500 ft, for example). To snap to the national grid, you need to modify the 'example.yml' file, either manually or in this code.

In [6]:
# we are not snapping to the national grid at this point in time
dx = 1320
dy = dx

In [7]:
xcells = int(np.ceil(tot_xdist/dx))
ycells = int(np.ceil(tot_ydist/dy))

In [8]:
xcells,ycells, xcells*ycells

(603, 275, 165825)

## <span class="anchor" id="yml">3. Modify the .yml file</span>
[Return to Top](#top)<br>

#### Load the example.yml file into Python. This is the generic .yml file that should remain unchanged as much as possible. 

In [9]:
with open('example.yml', 'r') as ifp:
    inyml = yaml.load(ifp, Loader=yaml.FullLoader)

#### Redefine the model offset, which defines the lower left corner of the model grid.

#### Define the appropriate coordinate system, in our case this is lambert. 

In [10]:
inyml['setup_grid']['xoff'] = bounds[0]
inyml['setup_grid']['yoff'] = bounds[1]
inyml['setup_grid']['crs'] = crs

#### Define the cell size, the number of layers, the number of rows, and the number of columns

In [11]:
inyml['dis']['griddata']['delr'] = dy
inyml['dis']['griddata']['delc'] = dx
inyml['dis']['dimensions']['nlay'] = 1
inyml['dis']['dimensions']['nrow'] = ycells
inyml['dis']['dimensions']['ncol'] = xcells

#### Write the updated information into the .yml file specifically created for the Mahomet Aquifer. 

In [13]:
with open('mahomet.yml', 'w') as ofp:
    yaml.dump(inyml, ofp)

## <span class="anchor" id="dis">4. Discretization Package</span>
[Return to Top](#top)<br>

#### In this block of code, we are creating the MODFLOW object using the updated 'mahomet.yml' file. We haven't assigned much yet, but this will create our spatial discretization object. 


In [14]:
m = mfsetup.MF6model.setup_from_yaml('mahomet.yml')

loading configuration file mahomet.yml...

Setting up mahomet model from data in None


validating configuration...
DIS package
done with validation.

setting up model grid...
wrote C:\Users\dbabrams\Documents\ISWS\GitHub\neversink_workflow_ISWSfork\notebooks_preprocessing_complete - Mahomet\tmp\mahomet\mahomet_grid.json
writing C:\Users\dbabrams\Documents\ISWS\GitHub\neversink_workflow_ISWSfork\notebooks_preprocessing_complete - Mahomet\tmp\mahomet\postproc\shps\mahomet_bbox.shp... Done
finished in 0.81s


Setting up TDIS package...
finished in 0.05s


Setting up IMS package...
finished in 0.00s


Setting up DIS package...
wrote .\external\top.dat, took 0.05s
loading original\mahomet_top.dat.original, shape=(275, 603), took 0.09s
computing cell thicknesses...
finished in 3.19s

wrote .\external\top.dat, took 0.05s
wrote .\external\botm_000.dat, took 0.03s
wrote .\external\top.dat, took 0.05s

reading C:\Users\dbabrams\Documents\ISWS\GitHub\neversink_workflow_ISWSfork\source_data\Shape

In [18]:
#just a quick checksum to make sure the code worked
m.dis.idomain.array.sum() * 5

829125

## <span class="anchor" id="shapefile">5. Create a shapefile of the model grid</span>
[Return to Top](#top)<br>

#### This code creates a shapefile of the model grid. Compare to the original 'mahomet_dissolve.shp' that we imported. If they are a match, then congratulations, you have taken your first step into creating a model with MODFLOW SETUP!!!!


In [16]:
m.dis.export('mahomet{}ft.dis.shp'.format(dx))

wrote mahomet1320ft.dis.shp


#### Before we move on, let's change our model back to the base directory. Note that this may fail if one of the above code blocks does not run properly

In [17]:
os.chdir(basedir)
